In [ ]:
import re
import json
from slugify import slugify

def read_bulletpoints(file, parse_data=True):
    merge_bullets = lambda b_list: '  \n'.join([ " - %s" % l for l in b_list])
    lines:list = None
    with open(file, "r") as r:
        lines = r.readlines()

    re_spaces = re.compile("^\s+.")

    bullets = { }
    current_ride:str = None
    bullet_list=[]
    for l in lines:
        if l.strip():
            if re_spaces.search(l): # Add bullets
                bullet_list.append(l.strip())
            else: # Value
                if current_ride:
                    bullets[current_ride]["description"] = merge_bullets(bullet_list)
                    bullet_list = []
                    
                data = l.split('|')
                current_ride = slugify(data[0].strip(), separator='_')
                bullets[current_ride] = {
                    "name": data[0].strip()
                }
                if parse_data:
                    location = [float(i) for i in data[1].split(',')]
                    bullets[current_ride]["location"] = location
    
    if current_ride:
        bullets[current_ride]["description"] = merge_bullets(bullet_list)
        bullet_list = []
        
    return bullets

## Rides:

In [ ]:
rides_descs = read_bulletpoints("_rides.txt", False)
# "expensive" search, let's waste cpu cycles, yey!

rides = None
with open("_rides.json", "r") as r:
    rides = json.load(r)
    
for r in rides:
    id_ = slugify(rides[r]["name"], separator='_')
    if id_ in rides_descs:
        rides[id_]['description'] = rides_descs[id_]["description"]
        
with open("_rides.json", "w", encoding='utf8') as w:
    json.dump(rides, w)

## Interesting people

In [ ]:
interesting = read_bulletpoints("_interesting.txt")

## Merge ifo

In [ ]:
debug = True

trips_json:dict = None
with open("../trip.json", "r") as r:
    trips_json = json.load(r)

trips_json["routes"] = {
    "type": "routes",
    "icon": "map",
    "name": "Moving around",
    "routes": rides
}

trips_json["people"] = {
    "type": "pins",
    "icon": "users",
    "name": "Interesting people",
    "pins": interesting
}
with open("../trip.json", "w") as w:
    json.dump(trips_json, w, indent=None if not debug else 4)